This notebook enables you to train a chess embedding from scratch on GPU in Google Colab!

# Setup

In [ ]:
# Install dependencies
!git clone https://github.com/patrickfrank1/chess-embedding-learning
!cd chess-embedding-learning && git checkout colab-training
!mv chess-embedding-learning/* /content/
!rm chess-embedding-learning -rf
%pip install -r requirements.txt
%pip install gdown

In [ ]:
# Download training data
!gdown --id 1_WnBplURUmIZDf3VAZmRGmI-QFJkjs2A
!tar -xvf tensor_format.tar.bz2 -C /content/data
!rm tensor_format.tar.bz2

# Zero code training

Train an embedding without writing any custom code. You can configure model hyperparameters in the code cell below.
A few different model architectures are available out of the box, including dense neural networks and convolutional neural networks.



In [6]:
%%writefile config.yaml
preprocess:
  pgn_path: ./data/pgn/lichess_db_standard_rated_2013-01.pgn
  save_path: ./data/tensors/lichess_db_standard_rated_2013-01_v2.h5
  game_filter: filter_out_bullet_games
  game_processor: positions_to_tensor
  chunk_size: 100000
  number_games: 100
train:
  data:
    train_dir: ./data/tensors/train
    test_dir: ./data/tensors/test
    train_batch_size: 256
    test_batch_size: 256
  model:
    save_dir: ./models
    train_steps_per_epoch: 20
    test_steps_per_epoch: 5
    loss: binary_crossentropy
    tf_callbacks: [checkpoints]
evaluate:
  data:
    model_dir: ./models
    test_dir: ./data/tensors/test
    test_batch_size: 64
  eval:
    number_examples: 10
    result_dir: ./results


Overwriting config.yaml


In [ ]:
# Train and evaluate model
!python train_model.py --config config.yaml | tee train.log

# Custom training

The cells below are a copy of the train_model.py script, but spread out and easily editable for convenience. You can use default function provided by the chesspos library or write your own.

## Model architecture

Provide an implementation for the `_define_encoder`, `_define_decoder`, and `_define_model` functions if you want to train an autoencoder. At least implement the `_define_model` function to train any neural network.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

from chesspos.models.autoencoder import AutoencoderModel

class CustomAutoencoder(AutoencoderModel):
	def __init__(self, **kwargs):
		self.embedding_size = 256
		super().__init__(**kwargs)

	def _model_helper(self) -> dict:
		encoder_input = layers.Input(shape=(8,8,15), dtype=tf.float16)
		encoder = layers.Reshape((8*8*15,))(encoder_input)
		encoder = layers.Dense(2*self.embedding_size, activation='relu')(encoder)
		encoder = layers.Dense(self.embedding_size, activation='relu')(encoder)

		decoder_input = layers.Input(shape=(self.embedding_size,))
		decoder = layers.Dense(2*self.embedding_size, activation='relu')(decoder_input)
		decoder = layers.Dense(8*8*15, activation='relu')(decoder_input)
		decoder = layers.Reshape((8,8,15))(decoder)

		encoder = keras.Model(inputs=encoder_input, outputs=encoder, name='encoder')
		decoder = keras.Model(inputs=decoder_input, outputs=decoder, name='decoder')
		autoencoder = keras.Model(inputs=encoder_input, outputs=decoder(encoder(encoder_input)), name='autoencoder')

		return {'encoder': encoder, 'decoder': decoder, 'autoencoder': autoencoder}

	def _define_encoder(self) -> Model:
		return self._model_helper()['encoder']

	def _define_decoder(self):
		return self._model_helper()['decoder']

	def _define_model(self) -> Model:
		return self._model_helper()['autoencoder']

## Training configuration

Customize the training setup and run the training. First define how the training tensors should be preprocessed for the neural network. The provided samples encode a chess position as tensor of shape (8, 8, 15). Each plane represents a (piece, color) combination. The 13th plane encodes the castling rights. The 14th plane encodes the en passant square. The 15th plane encodes the side to move.

Here we give a simple example on how to preprocess the training tensors for the network defines above.

In [ ]:
from typing import Tuple
from chess import Board
from chesspos.preprocessing import board_to_tensor, tensor_to_board
import numpy as np

The neural network needs the same sample as input and target.

In [ ]:
def custom_sample_preprocessor(samples: np.ndarray) -> Tuple[np.ndarray]:
	return tuple([samples, samples])

If we want to analyse the autoencoder encodings, we need to define how to convert chess boards into input tensors and vice versa. Here we basically need to wrap the sample in a batch with size 1.

In [ ]:
def custom_board_to_input(board: Board) -> np.ndarray:
	return np.expand_dims(board_to_tensor(board), axis=0)

def custom_tensor_to_board(tensor: np.ndarray) -> Board:
	return tensor_to_board(tensor[0])

Set up training run.

In [ ]:
%matplotlib inline
import chess
import yaml
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from chesspos.preprocessing import SampleGenerator

In [ ]:
train_params = yaml.safe_load(open('config.yaml'))['train']
evaluate_params = yaml.safe_load(open('config.yaml'))['evaluate']

data_params = train_params['data']
model_params = train_params['model']
evaluate_data_params = evaluate_params['data']
evaluate_eval_params = evaluate_params['eval']

train_generator = SampleGenerator(
	sample_dir = data_params['train_dir'],
	sample_preprocessor = custom_sample_preprocessor,
	batch_size = data_params['train_batch_size'],
	sample_type = np.float32,
)

test_generator = SampleGenerator(
	sample_dir = data_params['test_dir'],
	sample_preprocessor = custom_sample_preprocessor,
	batch_size = data_params['test_batch_size'],
	sample_type = np.float32,
)

autoencoder = CustomAutoencoder(
	train_generator=train_generator,
	test_generator=test_generator,
	train_steps_per_epoch=model_params['train_steps_per_epoch'],
	test_steps_per_epoch=model_params['test_steps_per_epoch'],
	save_dir=model_params['save_dir'],
	output_to_board = custom_tensor_to_board,
	board_to_input = custom_board_to_input,
	loss = model_params['loss'],
	tf_callbacks = model_params['tf_callbacks'] + [
		keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=10, verbose=1, mode='min', restore_best_weights=True),
		keras.callbacks.TensorBoard(log_dir=model_params['save_dir'], histogram_freq=1, write_images=True, embeddings_freq=1)
	]
)

Train the model.

In [ ]:
history = autoencoder.train()
autoencoder.save()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Evaluate a trained model

In [ ]:
autoencoder.load()
number_examples = evaluate_eval_params['number_examples']

Plot the samples with the lowest reconstruction loss

In [ ]:
autoencoder.plot_best_samples(number_samples=number_examples)

Plot the samples with the highest reconstruction loss

In [ ]:
autoencoder.plot_worst_samples(number_samples=number_examples)

Interpolate between two sample positions.

In [ ]:
board1 = chess.Board()
board2 = chess.Board("rnbq1rk1/ppp1bppp/4pn2/3p4/2PP4/5NP1/PP2PPBP/RNBQ1RK1 b - - 0 6")

sample1 = custom_board_to_input(board1)
sample2 = custom_board_to_input(board2)
autoencoder.interpolate(sample1=sample1, sample2=sample2)